## Challenge Exercise

Modify your solution to the challenge exercise in c_dataset.ipynb appropriately.

In [54]:
import datalab.bigquery as bq
import tensorflow as tf
import numpy as np
import pandas as pd
import shutil
import math
from google.datalab.ml import TensorBoard
print(tf.__version__)

1.8.0


In [55]:
df_train = pd.DataFrame(np.random.uniform(0.5, 2, 1000000), columns = ['radius'])
df_train['height'] = np.random.uniform(0.5, 2, 1000000)

df_valid = pd.DataFrame(np.random.uniform(0.5, 2, 10000), columns = ['radius'])
df_valid['height'] = np.random.uniform(0.5, 2, 10000)

In [56]:
labels = []
for index, row in df_train.iterrows():
  volume = math.pi * row['radius']**2 * row['height']
  labels.append(volume)
df_train['volume'] = labels

labels.clear()
for index, row in df_valid.iterrows():
  volume = math.pi * row['radius']**2 * row['height']
  labels.append(volume)
df_valid['volume'] = labels

# too long step, find another way to compute faster

In [57]:
noise = np.random.uniform(-1, 1, 1000000)
noise2 = np.random.uniform(-1, 1, 10000)
df_train['volume'] = round(df_train['volume'] + noise*0.1, 1)
df_train['radius'] = round(df_train['radius'], 1)
df_train['height'] = round(df_train['height'], 1)

df_valid['volume'] = round(df_valid['volume'] + noise2*0.1, 1)
df_valid['radius'] = round(df_valid['radius'], 1)
df_valid['height'] = round(df_valid['height'], 1)

In [58]:
df_train.to_csv('df_train.csv', header = False)
df_valid.to_csv('df_valid.csv', header = False)

In [59]:
CSV_COLUMNS = ['radius', 'height', 'volume', 'key']
LABEL_COLUMN = 'volume'
DEFAULTS = [[1.0], [1.0], [5.0], ['nokey']]

def read_dataset(filename, mode, batch_size = 512) :
  def decode_csv(value_column):
    columns = tf.decode_csv(value_column, record_defaults = DEFAULTS)
    features = dict(zip(CSV_COLUMNS, columns))
    label = features.pop(LABEL_COLUMN)
    return features, label
  
  filenames_dataset = tf.data.Dataset.list_files(filename)
  textlines_dataset = filenames_dataset.flat_map(tf.data.TextLineDataset)
  dataset = textlines_dataset.map(decode_csv)
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    num_epochs = None
    dataset = dataset.shuffle(buffer_size = 10 * batch_size)
  else:
    num_epochs = 1

  dataset = dataset.repeat(num_epochs).batch(batch_size)
  
  return dataset

In [60]:
INPUT_COLUMNS = [
  tf.feature_column.numeric_column('radius'),
  tf.feature_column.numeric_column('height'),
]

def add_more_features(feats):
  # nothing yet
  return feats

feature_cols = add_more_features(INPUT_COLUMNS)

In [61]:
def serving_input_fn():
    json_feature_placeholders = {
        'radius' : tf.placeholder(tf.float32, [None]),
        'height' : tf.placeholder(tf.float32, [None]),
    }
    # You can transforma data here from the input format to the format expected by your model.
    features = json_feature_placeholders # no transformation needed
    return tf.estimator.export.ServingInputReceiver(features, json_feature_placeholders)

In [62]:
def train_and_evaluate(output_dir, num_train_steps):
  estimator = tf.estimator.LinearRegressor(
                        model_dir = output_dir,
                        feature_columns = feature_cols)
  
  train_spec = tf.estimator.TrainSpec(
                        input_fn = lambda: read_dataset('./df_train.csv', mode = tf.estimator.ModeKeys.TRAIN),
                        max_steps = num_train_steps)

  exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)

  eval_spec = tf.estimator.EvalSpec(
                        input_fn = lambda: read_dataset('./df_valid.csv', mode = tf.estimator.ModeKeys.EVAL),
                        steps = None,
                        start_delay_secs = 1, # start evaluating after N seconds
                        throttle_secs = 10,  # evaluate every N seconds
                        exporters = exporter)
    
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [63]:
OUTDIR = './df_trained'
TensorBoard().start(OUTDIR)

TensorBoard was started successfully with pid 3927. Click here to access it.

3927

In [64]:
shutil.rmtree(OUTDIR, ignore_errors = True)
train_and_evaluate(OUTDIR, num_train_steps = 1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './df_trained', '_num_ps_replicas': 0, '_service': None, '_save_summary_steps': 100, '_is_chief': True, '_log_step_count_steps': 100, '_master': '', '_train_distribute': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_task_id': 0, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_session_config': None, '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb672e79b70>, '_global_id_in_cluster': 0, '_save_checkpoints_secs': 600}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 10 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow

In [65]:
TensorBoard().list()

,logdir,pid,port
0,./df_trained,3834,41671
1,./df_trained,3888,48591
2,./df_trained,3927,47977


In [66]:
pids_df = TensorBoard.list()
if not pids_df.empty:
  for pid in pids_df['pid']:
    TensorBoard().stop(pid)
    print('Stopped TensorBoard with pid {}'.format(pid))

Stopped TensorBoard with pid 3834
Stopped TensorBoard with pid 3888
Stopped TensorBoard with pid 3927


Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License